In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
filePath = '/home/amehdipour/Repos/CarSimCpp/Data/TIR_Templates/H_R20_18X6_6.tir'
temp = filePath.split('/')
temp2 = temp[-1].split('.')
tireName = temp2[0]
outPath = '/home/amehdipour/Repos/CarSimCpp/Data/ParsedTirCsv/' + tireName + '_Coeffs.csv'
outPath

'/home/amehdipour/Repos/CarSimCpp/Data/ParsedTirCsv/H_R25B_18X6_6_Coeffs.csv'

In [32]:
with open(filePath) as f:
    lines = f.readlines()

In [33]:
parametersToExtract = [
    # Longitudinal parameters
    "LCX", "LEX", "LHX", "LKX", "LMUX", "LUV", "LVX",
    "PEX1", "PEX2", "PEX3", "PEX4",
    "PCX1", "PDX1", "PDX2", "PDX3",
    "PKX1", "PKX2", "PKX3",
    "PHX1", "PHX2",
    "PVX1", "PVX2",
    "PPX1", "PPX2", "PPX3", "PPX4",

    # Lateral parameters
    "LCY", "LEY", "LHY", "LMUY", "LKY", "LKYC", "LMUV", "LVY",
    "PCY1",
    "PDY1", "PDY2", "PDY3",
    "PEY1", "PEY2", "PEY3", "PEY4", "PEY5",
    "PKY1", "PKY2", "PKY3", "PKY4", "PKY5", "PKY6", "PKY7",
    "PPY1", "PPY2", "PPY3", "PPY4",
    "PHY1", "PHY2", "PHY3",
    "PVY1", "PVY2", "PVY3", "PVY4",

    # Aligning torque parameters
    "LKZC", "LRES",
    "QBZ1", "QBZ2", "QBZ3", "QBZ4", "QBZ5", "QBZ10",
    "QCZ1",
    "QDZ1", "QDZ2", "QDZ3", "QDZ4", "QDZ6", "QDZ7", "QDZ8", "QDZ9", 'WIDTH=0.1905',

    "LXAL",
    "RBX1", "RBX2",
    "RCX1",
    "REX1", "REX2",
    "RHX1",

    # Lateral combined parameters
    "LYKA",
    "RBY1", "RBY2", "RBY3",
    "RCY1",
    "REY1", "REY2",
    "RHY1", "RHY2",
    "RVY1", "RVY2", "RVY3", "RVY4", "RVY5", "RVY6",

    # Normal load parameters
    "LCZ", "PZ1", "LFZO", "FNOMIN",

    # Overturning moment parameters
    "QSX1", "QSX2", "QSX3",

    # Rolling resistance parameters
    "LMY", "QSY1", "QSY2",

    # Self-aligning combined parameters
    "SSZ1", "SSZ2", "SSZ3", "SSZ4",

    # Turn slip parameters
    "PDYP1", "PDYP2", "PDYP3", "PDYP4", "PKYP1",

    # Tire properties"
    "MASS", "IYY", "IXX", "UNLOADED_RADIUS", "WIDTH", "ASPECT_RATIO", "RIM_RADIUS", "RIM_WIDTH", "LONGVL" ]

In [34]:
outputs = []
flagSkipSection = False
sectionsToSkip = ['[UNITS]', '[CONTACT_PATCH]']
for idx, line in enumerate(lines):
    if (any(section in line for section in sectionsToSkip)):
        print(line)
        flagSkipSection = True
        continue
    if flagSkipSection:
        if ']' in line:
            flagSkipSection = False
        else:
            continue
    newLine = line.strip()
    newLine = newLine.replace(' ','')
    newLine = newLine.split('$')[0]
    lines[idx] = newLine
    for coeff in parametersToExtract:
        if coeff in newLine:
            outputs.append(newLine)
# lines

[UNITS]

[CONTACT_PATCH]



In [35]:
# Assuming 'parameters_to_extract' is your list of desired parameters
# and 'output' is your list of lines from the .tir file

# Step 1: Find which parameters are present in the output lines
found_params = set()
for param in parametersToExtract:
    # Check if this parameter exists in any line
    found = False
    for line in outputs:
        # Check if the parameter appears at the start of a line followed by whitespace and equals sign
        if line.strip().startswith(param + " ") or line.strip().startswith(param + "="):
            found = True
            break

    if found:
        found_params.add(param)

# Step 2: Find which parameters were not found in any line
missing_params = set(parametersToExtract) - found_params

# Print the missing parameters
if missing_params:
    print(f"Found {len(found_params)} parameters out of {len(parametersToExtract)} requested.")
    print("The following parameters were not found in the .tir file:")
    for param in sorted(missing_params):
        print(f"  - {param}")
else:
    print("All requested parameters were found in the .tir file.")

Found 123 parameters out of 129 requested.
The following parameters were not found in the .tir file:
  - LCZ
  - LMUV
  - LUV
  - PHY3
  - PZ1
  - WIDTH=0.1905


In [36]:
lines

['[MDI_HEADER]',
 "FILE_TYPE='tir'",
 'FILE_VERSION=3',
 "FILE_FORMAT='ASCII'",
 '',
 '[UNITS]\n',
 "LENGTH                   =    'meter'        \n",
 "FORCE                    =    'newton'       \n",
 "ANGLE                    =    'radians'      \n",
 "MASS                     =    'kg'           \n",
 "TIME                     =    'second'       \n",
 '$---------------------------------------------------------------------Model\n',
 '[MODEL]',
 'FITTYP=61',
 "TYRESIDE='Left'",
 'LONGVL=11.1',
 'VXLOW=1',
 'ROAD_INCREMENT=0.01',
 'ROAD_DIRECTION=1',
 "PROPERTY_FILE_FORMAT='USER'",
 'USER_SUB_ID=815',
 'N_TIRE_STATES=4',
 'USE_MODE=114',
 'HMAX_LOCAL=0.00025',
 'TIME_SWITCH_INTEG=0.1',
 '',
 '[DIMENSION]',
 'UNLOADED_RADIUS=0.26',
 'WIDTH=0.1905',
 'ASPECT_RATIO=0.55',
 'RIM_RADIUS=0.127',
 'RIM_WIDTH=0.203',
 '',
 '[OPERATING_CONDITIONS]',
 '!',
 '!NevermodifyNOMPRES,tochangetheinflationpressureofthetyremodifyINFLPRESonly.',
 '!',
 'INFLPRES=85000',
 'NOMPRES=83000',
 '',
 '[INERTI

In [37]:
outputDict = {}
for idx, val in enumerate(outputs):
    paramName = val.split('=')[0]
    paramValue = float(val.split('=')[1])
    outputDict.update({paramName: paramValue})
# df = pd.DataFrame.from_dict(outputDict).T
# df.to_csv(outputDict)
# df

In [38]:
outputDict['MASS']

9.3

In [39]:
df = pd.DataFrame.from_dict(outputDict, orient='index', columns=['Value'])
df.to_csv(outPath)
print(f"Output saved to {outPath}")

Output saved to /home/amehdipour/Repos/CarSimCpp/Data/ParsedTirCsv/H_R25B_18X6_6_Coeffs.csv
